In [6]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pysd
import pyNetLogo
import time

In [7]:
listofcolumnsresults = ['"Extraction capacity in preparation - Gas"', 
                        '"Extraction capacity installed - Gas"',
                        '"Mothballed capacity - Gas"',
                        '"Undiscovered resources - Gas"',
                        '"Discovered resources reserve base - Gas"',
                        '"Extraction capacity in preparation - Oil"',
                        '"Extraction capacity installed - Oil"',
                        '"Mothballed capacity - Oil"',
                        '"Energy stocks - Oil"',
                        '"Undiscovered resources - Oil"',
                        '"Discovered resources reserve base - Oil"',
                        '"Extraction capacity in preparation - Coal"',
                        '"Extraction capacity installed - Coal"',
                        '"Mothballed capacity - Coal"',
                        '"Energy stocks - Coal"',
                        '"Undiscovered resources - Coal"',
                        '"Discovered resources reserve base - Coal"',
                        '"Extraction capacity in preparation- Nuclear"',
                        '"Extraction capacity installed- Nuclear"',
                        '"Mothballed capacity- Nuclear"',
                        '"Extraction capacity in preparation - Biofuels"',
                        '"Extraction capacity installed - Biofuels"',
                        '"Mothballed capacity - Biofuels"',
                        '"Energy stocks - Biofuels"',
                        '"Extraction capacity in preparation - other Renewables"',
                        '"Extraction capacity installed - other Renewables"',
                        '"Mothballed capacity - other Renewables"',
                        'GDP',
                        'Energy Intensity GDP',
                        'Future Effect of Shortage on Decoupling',
                        'Effect of Supply Shortage on Decoupling',
                        'TIME']

listofcolumnsinput = ['"Initial extraction capacity in preparation - Gas"', 
                        '"Initial extraction capacity - Gas"',
                        '"Initial mothballed capacity - Gas"',
                        '"Initial Undiscovered resources - Gas"',
                        '"Initial reserve base - Gas"',
                        '"Initial extraction capacity in preparation - Oil"',
                        '"Initial extraction capacity - Oil"',
                        '"Initial mothballed capacity - Oil"',
                        '"Initial Energy stocks - Oil"',
                        '"Initial Undiscovered resources - Oil"',
                        '"Initial reserve base - Oil"',
                        '"Initial extraction capacity in preparation - Coal"',
                        '"Initial extraction capacity - Coal"',
                        '"Initial mothballed capacity - Coal"',
                        '"Initial Energy stocks - Coal"',
                        '"Initial Undiscovered resources - Coal"',
                        '"Initial reserve base - Coal"',
                        '"Initial extraction capacity in preparation- Nuclear"',
                        '"Initial extraction capacity- Nuclear"',
                        '"Initial mothballed capacity- Nuclear"',
                        '"Initial extraction capacity in preparation - Biofuels"',
                        '"Initial extraction capacity - Biofuels"',
                        '"Initial mothballed capacity - Biofuels"',
                        '"Initial Energy stocks - Biofuels"',
                        '"Initial extraction capacity in preparation - other Renewables"',
                        '"Initial extraction capacity - other Renewables"',
                        '"Initial mothballed capacity - other Renewables"',
                        'Initial GDP',
                        'Initial Energy Intensity GDP',
                        'Initial Shortage Effect on Decoupling',
                        'Initial Effect of Supply Shortage on Decoupling',
                        'INITIAL TIME']

# Without functions

In [ ]:
dealtime = 10
x = dealtime*16
sd_model = pysd.read_vensim("modelV15.mdl")
modelinput = pd.read_csv("testlanden.csv")
ncountries = len(modelinput)
inputdict = modelinput.to_dict('index')
resultsnew = []
for value in inputdict.values():
    value['FINAL TIME'] = 2010 + dealtime
    resultsnew.append(sd_model.run(params=value))

In [ ]:
finalvalues = []
for singleresult in resultsnew:
    finalvalues.append(singleresult.ix[[x], listofcolumnsresults])

In [ ]:
dictlist = []
for valueset in finalvalues:
    newdict = valueset.to_dict('records')[0]
    for k_results, k_input in zip(listofcolumnsresults, listofcolumnsinput):
        newdict[k_input] = newdict.pop(k_results)
    dictlist.append(newdict)

In [ ]:
dictdict = {}
for i in range(ncountries):
    dictdict.update({i:{}})

dictdict = {key:dictlist[i] for (key, value), i in zip(dictdict.items(), range(ncountries))}

In [ ]:
resultsnew2 = []
for value in dictdict.values():
    value['FINAL TIME'] = value['INITIAL TIME'] + dealtime
    resultsnew2.append(sd_model.run(params=value))

In [ ]:
combresults = []
combresults.append(resultsnew)
combresults.append(resultsnew2)

results = [pd.concat([entry[n] for entry in combresults]) for n in range(ncountries)]

# With functions

In [ ]:
def runmodelsnoloop(years, dealtime):
    
    sd_model = pysd.read_vensim("modelV15.mdl")
    modelinput = pd.read_csv("testlanden.csv")

    ncountries = len(modelinput)
    x = dealtime*16
    inputdict = modelinput.to_dict('index')

    resultsnew = []
    for value in inputdict.values():
#         value['INITIAL TIME'] = inittime
        value['FINAL TIME'] = 2010 + dealtime
        resultsnew.append(sd_model.run(params=value))
    
#     Loop start
    
    finalvalues = []
    for singleresult in resultsnew:
        finalvalues.append(singleresult.ix[[x], listofcolumnsresults])
    
    dictlist = []
    for valueset in finalvalues:
        newdict = valueset.to_dict('records')[0]
        for k_results, k_input in zip(listofcolumnsresults, listofcolumnsinput):
            newdict[k_input] = newdict.pop(k_results)
        dictlist.append(newdict)
    
    dictdict = {}
    for i in range(ncountries):
        dictdict.update({i:{}})
        
#         print(i)
        
#     print(len(dictlist))
    
    dictdict = {key:dictlist[i] for (key, value), i in zip(dictdict.items(), range(ncountries))}
    
    resultsnew2 = []
    for value in dictdict.values():
        value['FINAL TIME'] = value['INITIAL TIME'] + dealtime
        resultsnew2.append(sd_model.run(params=value))
        
    combresults = []
    combresults.append(resultsnew)
    combresults.append(resultsnew2)

    results = [pd.concat([entry[n] for entry in combresults]) for n in range(ncountries)]
    
    return results

In [ ]:
def runmodelswithloop(years, dealtime):
    
    sd_model = pysd.read_vensim("modelV15.mdl")
    modelinput = pd.read_csv("testlanden.csv")

    ncountries = len(modelinput)
    x = dealtime*16
    inputdict = modelinput.to_dict('index')

    resultsnew = []
    for value in inputdict.values():
        value['FINAL TIME'] = 2010 + dealtime
        resultsnew.append(sd_model.run(params=value))
    
    combresults = [resultsnew]
    for n in range(dealtime, years, dealtime):
    
        finalvalues = []
        for singleresult in resultsnew:
            finalvalues.append(singleresult.ix[[x], listofcolumnsresults])
    
        dictlist = []
        for valueset in finalvalues:
            newdict = valueset.to_dict('records')[0]
            for k_results, k_input in zip(listofcolumnsresults, listofcolumnsinput):
                newdict[k_input] = newdict.pop(k_results)
            dictlist.append(newdict)
    
        dictdict = {}
        for i in range(ncountries):
            dictdict.update({i:{}})

        dictdict = {key:dictlist[i] for (key, value), i in zip(dictdict.items(), range(ncountries))}
    
        resultsnew = []
        for value in dictdict.values():
            value['FINAL TIME'] = value['INITIAL TIME'] + dealtime
            resultsnew.append(sd_model.run(params=value))
        
        combresults.append(resultsnew)

    results = [pd.concat([entry[n] for entry in combresults]) for n in range(ncountries)]
    
    return results

In [8]:
def runnetlogo(agent_gas, netlogo):
    netlogo.write_NetLogo_attriblist(agent_gas[['who','GS', 'PP']], 'country')
    netlogo.command('repeat 1 [go]')
    
    Gasimport = []
    Gasexport = []
    
    Gasimport.append(netlogo.report('importA'))
    Gasimport.append(netlogo.report('importB'))
    Gasimport.append(netlogo.report('importC'))
    Gasimport.append(netlogo.report('importD'))
    Gasimport.append(netlogo.report('importE'))
    Gasimport.append(netlogo.report('importF'))
    
    Gasexport.append(netlogo.report('exportA'))
    Gasexport.append(netlogo.report('exportB'))
    Gasexport.append(netlogo.report('exportC'))
    Gasexport.append(netlogo.report('exportD'))
    Gasexport.append(netlogo.report('exportE'))
    Gasexport.append(netlogo.report('exportF'))
    
    return Gasimport, Gasexport

In [9]:
def hybridloop(years, dealtime):
    
    sd_model = pysd.read_vensim("modelV15.mdl")
    netlogo = pyNetLogo.NetLogoLink(gui=False)
    netlogo.load_model(r'modelv21.nlogo')
    netlogo.command('setup')
    modelinput = pd.read_csv("testlanden.csv")

    ncountries = len(modelinput)
    x = dealtime*16
    inputdict = modelinput.to_dict('index')

    resultsnew = []
    for value in inputdict.values():
        value['FINAL TIME'] = 2010 + dealtime
        resultsnew.append(sd_model.run(params=value))
    
    combresults = [resultsnew]
    for n in range(dealtime, years, dealtime):
    
        GS = []
        for country in resultsnew:
            GS.append(country['Gas to trade'].values[x])
            
        gasdata = {'who': [0, 1, 2, 3, 4, 5],
                   'GS': GS,
                   'PP': [9, 5, 9, 10, 4, 6]}
    
        agent_gas = pd.DataFrame (gasdata, columns=['who', 'GS', 'PP'])
        
        Gasimport, Gasexport = runnetlogo(agent_gas, netlogo)    
    
        finalvalues = []
        for singleresult in resultsnew:
            finalvalues.append(singleresult.ix[[x], listofcolumnsresults])
    
        dictlist = []
        for valueset in finalvalues:
            newdict = valueset.to_dict('records')[0]
            for k_results, k_input in zip(listofcolumnsresults, listofcolumnsinput):
                newdict[k_input] = newdict.pop(k_results)
            dictlist.append(newdict)
    
        dictdict = {}
        for i in range(ncountries):
            dictdict.update({i:{}})

        dictdict = {key:dictlist[i] for (key, value), i in zip(dictdict.items(), range(ncountries))}
    
        resultsnew = []
        for value, gi, ge in zip(dictdict.values(), Gasimport, Gasexport):
            value['FINAL TIME'] = value['INITIAL TIME'] + dealtime
            value['Gas Import'] = gi
            value['Gas Export'] = ge
            resultsnew.append(sd_model.run(params=value))
        
        combresults.append(resultsnew)

    results = [pd.concat([entry[n] for entry in combresults]) for n in range(ncountries)]
    
    return results

In [10]:
starttime = time.time()
results = hybridloop(40, 5)
endtime = time.time()
elapsedtime = endtime - starttime
elapsedtime

IncompleteParseError: Rule 'rules' matched in its entirety, but it didn't consume all the text. The non-matching portion of the text begins with '\ costs\ relative\ i' (line 20, column 25).

In [ ]:
results[2]['Gas Import'].plot()
results[2]['Gas Export'].plot()

In [ ]:
inputdictest = inputdict
x = 10
for value in inputdictest.values():
    value['FINAL TIME'] = 2010
    resultsnew.append(sd_model.run(params=value))
    
#     Loop start
    
finalvalues = []
for singleresult in resultsnew:
    finalvalues.append(singleresult.ix[[80], listofcolumnsresults])

In [ ]:
a = time.time()
c = 1+3
b = time.time()
elapsedtime = b - a
elapsedtime